In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import chess
import h5py
import chess.pgn

Using TensorFlow backend.


In [2]:
pgn = open("2004Data.pgn")

In [3]:
def position_list_one_hot(self):
    builder = []
    builder_append = builder.append
    for square in chess.SQUARES_180:
        mask = chess.BB_SQUARES[square]

        if not self.occupied & mask:
            builder.extend([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        elif bool(self.occupied_co[chess.WHITE] & mask):
            if self.pawns & mask:
                builder.extend([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
            elif self.knights & mask:
                builder.extend([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
            elif self.bishops & mask:
                builder.extend([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
            elif self.rooks & mask:
                builder.extend([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
            elif self.queens & mask:
                builder.extend([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
            elif self.kings & mask:
                builder.extend([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
        elif self.pawns & mask:
            builder.extend([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        elif self.knights & mask:
            builder.extend([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        elif self.bishops & mask:
            builder.extend([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        elif self.rooks & mask:
            builder.extend([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
        elif self.queens & mask:
            builder.extend([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
        elif self.kings & mask:
            builder.extend([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

    return np.reshape(np.array(builder), (1, 768))

def position_list(self):
    '''same as position_list_one_hot except this is converts pieces to
    numbers between 1 and 12. Used for piece_moved function'''
    builder = []
    builder_append = builder.append
    for square in chess.SQUARES_180:
        mask = chess.BB_SQUARES[square]

        if not self.occupied & mask:
            builder_append(0)
        elif bool(self.occupied_co[chess.WHITE] & mask):
            if self.pawns & mask:
                builder_append(7)
            elif self.knights & mask:
                builder_append(8)
            elif self.bishops & mask:
                builder_append(9)
            elif self.rooks & mask:
                builder_append(10)
            elif self.queens & mask:
                builder_append(11)
            elif self.kings & mask:
                builder_append(12)
        elif self.pawns & mask:
            builder_append(1)
        elif self.knights & mask:
            builder_append(2)
        elif self.bishops & mask:
            builder_append(3)
        elif self.rooks & mask:
            builder_append(4)
        elif self.queens & mask:
            builder_append(5)
        elif self.kings & mask:
            builder_append(6)

    return np.reshape(np.array(builder), (1, 64))

chess.BaseBoard.position_list_one_hot = position_list_one_hot
chess.BaseBoard.position_list = position_list

In [4]:
boardstate = []
while True:
  game = chess.pgn.read_game(pgn)
  if game is None:
            break
  board = game.board()
  for move in game.mainline_moves():
     curBoard = board.position_list()
     curOneHot = board.position_list_one_hot()
     board.push(move)
     aftBoard = board.position_list()
     aftOneHot = board.position_list_one_hot()
    
     boardstate.append((curBoard, aftBoard, curOneHot, aftOneHot))

In [5]:
df = pd.DataFrame(boardstate)

In [6]:
df["input"] = df[0].apply(lambda x: x[0] )
df["output"] = df[1].apply(lambda x: x[0] )
df["inputOneHot"] = df[2].apply(lambda x: x[0] )
df["outputOneHot"] = df[3].apply(lambda x: x[0] )

In [7]:
data = df[["input", "output", "inputOneHot","outputOneHot"]]

In [8]:
def piece_moved(position1, position2):
    affected_squares = []
    for i in range(64):  # Step 1
        if position1[i] != position2[i]:
            affected_squares.append(i)
    if len(affected_squares) > 2:  # Step 2
        for square in affected_squares:
            if position1[square] == 12 or position1[square] == 6:  # Step 2a
                moved_from = square
            if position2[square] == 12 or position2[square] == 6:
                moved_to = square
            if position1[square] == 0:  # Step 2b
                if position2[square] == 1:
                    moved_to = square
                    for square in affected_squares:
                        if position1[square] == 1:
                            moved_from = square
                elif position2[square] == 7:
                    moved_to = square
                    for square in affected_squares:
                        if position1[square] == 7:
                            moved_from = square
    else:
        if position2[affected_squares[0]] == 0:
            moved_from, moved_to = affected_squares[0], affected_squares[1]
        else:
            moved_from, moved_to = affected_squares[1], affected_squares[0]
    return moved_from, moved_to

In [9]:
data["From_To"] = data.apply(lambda x: piece_moved(x[0],x[1]),axis=1)

/home/nathan_a_officer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
data["From"] = data["From_To"].apply(lambda x: x[0])
data["To"] = data["From_To"].apply(lambda x: x[1])

/home/nathan_a_officer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/nathan_a_officer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
data

,input,output,inputOneHot,outputOneHot,From_To,From,To
0,"[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 1, 1, 1, 1, 1, ...","[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","(52, 36)",52,36
1,"[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 1, 1, 1, 1, 1, ...","[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","(10, 26)",10,26
2,"[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 0, 1, 1, 1, 1, ...","[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","(62, 45)",62,45
3,"[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 0, 1, 1, 1, 1, ...","[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 0, 0, 1, 1, 1, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","(11, 19)",11,19
4,"[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 0, 0, 1, 1, 1, ...","[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 0, 0, 1, 1, 1, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","(51, 35)",51,35
...,...,...,...,...,...,...,...
2537143,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(11, 17)",11,17
2537144,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(36, 26)",36,26
2537145,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(24, 25)",24,25
2537146,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(26, 43)",26,43


In [12]:
def makeReshape(x):
  remake = []
  for y in x:
    yz = y.reshape(8,8,12)
    remake.extend(y.reshape(8,8,12))
  return np.reshape(np.array(remake), (2537148,8,8,12))

In [13]:
dfxm = makeReshape(df["inputOneHot"])

In [14]:
FromNP = data["From"].to_numpy()
ToNP = data["To"].to_numpy()

In [15]:
#np.save("input.npy", dfxm)

In [16]:
#np.save("from.npy", FromNP)
#np.save("to.npy", ToNP)

In [2]:
input = np.load("input.npy")

In [4]:
fromY = np.load("from.npy")

In [78]:
toY = np.load("to.npy")

In [5]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(input,fromY, random_state=42, test_size=0.3)

In [6]:
from keras import backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import (Activation, Conv2D, Dense, Dropout, Flatten,
                          LeakyReLU, MaxPooling2D)
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD, Adadelta, Adam, RMSprop
K.clear_session()

batch_size = 20000
num_classes = 64
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)
epochs = 10
img_rows, img_cols = 8, 8
input_shape = (img_rows, img_cols, 12)

model = Sequential()
model.add(Conv2D(128, kernel_size=(2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(128, kernel_size=(2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])

In [7]:
#From Model

model.fit(x_train, y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test_cat), 
          verbose=1)

Train on 1776003 samples, validate on 761145 samples
Epoch 1/10
1776003/1776003 [==============================] - 776s 437us/step - loss: 2.9064 - accuracy: 0.1708 - val_loss: 2.3580 - val_accuracy: 0.2431
Epoch 2/10
1776003/1776003 [==============================] - 774s 436us/step - loss: 2.3321 - accuracy: 0.2479 - val_loss: 2.2180 - val_accuracy: 0.2763
Epoch 3/10
1776003/1776003 [==============================] - 773s 435us/step - loss: 2.2331 - accuracy: 0.2719 - val_loss: 2.1732 - val_accuracy: 0.2856
Epoch 4/10
1776003/1776003 [==============================] - 774s 436us/step - loss: 2.1774 - accuracy: 0.2870 - val_loss: 2.1233 - val_accuracy: 0.2990
Epoch 5/10
1776003/1776003 [==============================] - 777s 438us/step - loss: 2.1314 - accuracy: 0.2994 - val_loss: 2.0993 - val_accuracy: 0.3085
Epoch 6/10
1776003/1776003 [==============================] - 777s 437us/step - loss: 2.0973 - accuracy: 0.3095 - val_loss: 2.0748 - val_accuracy: 0.3147
Epoch 7/10
1776003/1776

In [8]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [9]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [11]:
from keras.models import model_from_json

In [12]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [13]:
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [17]:
prediction = loaded_model.predict(x_test)

In [59]:
predList = list(prediction[1])

In [66]:
WhereFrom = {}
values = []
for item in predList:
    num = format(item, '.12f')
    numIndex = predList.index(item)
    WhereFrom.update({numIndex:num})
    values.append(num)
    

In [70]:
values

['0.502145469189',
 '0.150383040309',
 '0.061777994037',
 '0.050007473677',
 '0.049716029316',
 '0.038870126009',
 '0.024137899280',
 '0.023295812309',
 '0.022204082459',
 '0.010841922835',
 '0.010417195968',
 '0.007836649194',
 '0.007615635637',
 '0.006734543946',
 '0.006267379038',
 '0.004826410674',
 '0.004483061843',
 '0.003964448348',
 '0.003887946950',
 '0.002423611702',
 '0.001682027942',
 '0.001293026144',
 '0.000765372242',
 '0.000760051829',
 '0.000531636295',
 '0.000498544308',
 '0.000393723894',
 '0.000257135835',
 '0.000237868473',
 '0.000150338994',
 '0.000143482801',
 '0.000112790767',
 '0.000106734595',
 '0.000106222702',
 '0.000088008906',
 '0.000086764849',
 '0.000080597165',
 '0.000069352391',
 '0.000065564600',
 '0.000058926515',
 '0.000054183911',
 '0.000052159365',
 '0.000050556475',
 '0.000049862268',
 '0.000049241236',
 '0.000046084901',
 '0.000037315567',
 '0.000035630950',
 '0.000035604418',
 '0.000027360651',
 '0.000025209589',
 '0.000024645742',
 '0.00002359

In [74]:
def get_key(val,the_dict): 
    for key, value in the_dict.items(): 
         if val == value: 
             return key 

In [79]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
x_trainTo, x_testTo, y_trainTo, y_testTo = train_test_split(input,toY, random_state=42, test_size=0.3)

In [80]:
y_trainTo_cat = keras.utils.to_categorical(y_train, num_classes)
y_testTo_cat = keras.utils.to_categorical(y_test, num_classes)

In [81]:
model.fit(x_trainTo, y_trainTo_cat,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_testTo, y_testTo_cat), 
          verbose=1)

Train on 1776003 samples, validate on 761145 samples
Epoch 1/10
1776003/1776003 [==============================] - 779s 438us/step - loss: 1.9661 - accuracy: 0.3489 - val_loss: 1.9987 - val_accuracy: 0.3370
Epoch 2/10
1776003/1776003 [==============================] - 778s 438us/step - loss: 1.9468 - accuracy: 0.3549 - val_loss: 1.9881 - val_accuracy: 0.3398
Epoch 3/10
1776003/1776003 [==============================] - 777s 438us/step - loss: 1.9271 - accuracy: 0.3614 - val_loss: 1.9803 - val_accuracy: 0.3422
Epoch 4/10
1776003/1776003 [==============================] - 774s 436us/step - loss: 1.9079 - accuracy: 0.3671 - val_loss: 1.9771 - val_accuracy: 0.3449
Epoch 5/10
1776003/1776003 [==============================] - 793s 446us/step - loss: 1.8873 - accuracy: 0.3734 - val_loss: 1.9669 - val_accuracy: 0.3486
Epoch 6/10
1776003/1776003 [==============================] - 788s 444us/step - loss: 1.8681 - accuracy: 0.3796 - val_loss: 1.9657 - val_accuracy: 0.3488
Epoch 7/10
1776003/1776

In [82]:
to_model_json = model.to_json()
with open("Tomodel.json", "w") as json_file:
    json_file.write(to_model_json)

In [83]:
model.save_weights("Tomodel.h5")
print("Saved model to disk")

Saved model to disk
